In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
# print(os.getcwd())
# sys.path.append(os.getcwd())

import logging
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn as nn
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: True


In [2]:
datadir= Path('/dlab/ldrive/CBT/USLJ-DSDE_DATA-I10008/BenchmarkDatasets/hpa-single-cell-image-classification')
train_dataset_dir = datadir.joinpath('train')
train_csv = datadir.joinpath('train_select_1K.csv')

In [3]:
from func.HPASCDataset import HPASCDataset, TrDataset
from func.trainer import load_train_objs
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Resize
input_ch_ct = 4
debug_size = 100
n_class = 19

transform_dataset = Compose(
    [
        ToTensor(), 
        Resize(size = (1024, 1024), antialias = True),
    ]

)
HPA_dataset = HPASCDataset(
                        input_csv = train_csv, 
                        root = train_dataset_dir, 
                        input_ch_ct = input_ch_ct,
                        n_class = n_class, 
                        debug_size = debug_size,
                        transform = transform_dataset
                        )

dataloader = DataLoader(
                    HPA_dataset,
                    batch_size=32,
                    pin_memory=True,
                    num_workers=4,
                    shuffle=False)

In [6]:
def get_mean_and_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for data in tqdm(loader):
        images = data['image']
        image_count_in_a_batch = images.size(0)
        print(images.shape)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        print(images.shape)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count_in_a_batch
    
    mean /= total_images_count
    std /= total_images_count
    return mean, std

In [7]:
get_mean_and_std(dataloader)

  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([32, 4, 1024, 1024])
torch.Size([32, 4, 1048576])


 25%|██▌       | 1/4 [00:12<00:38, 12.80s/it]

torch.Size([32, 4, 1024, 1024])
torch.Size([32, 4, 1048576])


 50%|█████     | 2/4 [00:16<00:15,  7.73s/it]

torch.Size([32, 4, 1024, 1024])
torch.Size([32, 4, 1048576])


100%|██████████| 4/4 [00:18<00:00,  3.05s/it]

torch.Size([4, 4, 1024, 1024])
torch.Size([4, 4, 1048576])


100%|██████████| 4/4 [00:18<00:00,  4.70s/it]


(tensor([0.0462, 0.0454, 0.0728, 0.0717]),
 tensor([0.1259, 0.0767, 0.1178, 0.1117]))